In [92]:
# importing the libraries
import pandas as pd
import os
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
from heapq import nlargest    
nlp = spacy.load("nl_core_news_sm")


In [93]:

def summarize_doc():
    
    """summarize the content of the document
    """
    mypath = r"D:\Arai4_Projects\kpmgProject\data\nl_txt"
    myfiles = os.listdir(mypath)
    stopwords = list(STOP_WORDS) # Build a List of Stopwords
    list_summary = []
    count =0

    for filename in os.listdir(mypath):
        if filename.endswith(".txt"):
            count = count+1
            text_path = mypath + "/" + filename
            with open(text_path, 'r', encoding='utf8') as f:   # open the file and read the text
                text = f.read()
                docx = nlp(text)   # Build an NLP Object
                
                mytokens = [token.text for token in docx]   # Tokenization of Text
                word_frequencies = {}   # Build Word Frequency
                for word in docx:
                    if word.text not in stopwords:
                            if word.text not in word_frequencies.keys():
                                word_frequencies[word.text] = 1
                            else:
                                word_frequencies[word.text] += 1
                maximum_frequency = max(word_frequencies.values())  # Maximum Word Frequency
                for word in word_frequencies.keys():  
                    word_frequencies[word] = (word_frequencies[word]/maximum_frequency)
            
                sentence_list = [ sentence for sentence in docx.sents ]  # Sentence Tokens
                sentence_scores = {}   # Sentence Score via comparrng each word with sentence
                for sent in sentence_list:  
                        for word in sent:
                            if word.text.lower() in word_frequencies.keys():
                                if len(sent.text.split(' ')) < 30:
                                    if sent not in sentence_scores.keys():
                                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                                    else:
                                        sentence_scores[sent] += word_frequencies[word.text.lower()]
                # Sentence Score Table
                sentence_scores 
                summarized_sentences = nlargest(7, sentence_scores, key=sentence_scores.get)
                summarized_sentences      
                final_sentences = [ w.text for w in summarized_sentences ] 
                summary = ' '.join(final_sentences)
            
                list_summary.append({'filename':re.split('_',filename)[0] + '.pdf', 'summary':summary})
                
    return list_summary

In [91]:
def merge_column():
    """merge the summary column to main dataframe
    """
    df = pd.read_csv(r"D:\Arai4_Projects\kpmgProject\data\B\metadata_with_fr_nl_info.csv")
    
    df_summary = pd.DataFrame(summarize_doc())
    df_merged = pd.merge(df, df_summary, on = 'filename')
    df_merged.to_csv("D:\Arai4_Projects\kpmgProject\data\B\metadata_with_fr_nl_info_summary.csv", index = False, header = True )

    return df_merged

df = merge_column()